# Ensemble Models

We performed ensembling with soft voting of seven models to improve our highest test accuracy.

The models can be downloaded from [Google Drive](https://drive.google.com/drive/folders/1F2isfUMEt7I6WH9LGIwc-IG-C2-9PWk4?usp=sharing)

## Import Dependencies 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.python.lib.io import file_io

%matplotlib inline

import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from sklearn.metrics import *
from keras.engine import Model
from keras.layers import Input, Flatten, Dense, Activation, Conv2D, MaxPool2D, BatchNormalization, Dropout, MaxPooling2D

import skimage
from skimage.transform import rescale, resize

In [ ]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

print('Importing successfully!')
print('tensorflow',tf.__version__)
print('tensorflow.keras',tf.keras.__version__)
print('keras',keras.__version__)
print('GPU',tf.config.list_physical_devices('GPU'))

## Import datasets

In [ ]:
Resize_pixelsize = 197

# Function that reads the data from the csv file, increases the size of the images and returns the images and their labels
def get_data(dataset_path, Resize_pixelsize = 197):
    
    file_stream = file_io.FileIO(dataset_path, mode='r')
    data = pd.read_csv(file_stream)
    data['pixels'] = data['pixels'].apply(lambda x: [int(pixel) for pixel in x.split()])
    X, Y = data['pixels'].tolist(), data['emotion'].values
    X = np.array(X, dtype='float32').reshape(-1,48,48,1)
    X = X/255.0
   
    X_res = np.zeros((X.shape[0], Resize_pixelsize,Resize_pixelsize,3))
    for ind in range(X.shape[0]): 
        sample = X[ind]
        sample = sample.reshape(48, 48)
        image_resized = resize(sample, (Resize_pixelsize, Resize_pixelsize), anti_aliasing=True)
        X_res[ind,:,:,:] = image_resized.reshape(Resize_pixelsize,Resize_pixelsize,1)

    Y_res = np.zeros((Y.size, 7))
    Y_res[np.arange(Y.size),Y] = 1    
    
    return  X, X_res, Y_res

dev_dataset_dir = '/Users/ouyang/Documents/GitHub/ST7_FER_Projet/FER_Dataset/csv/dev.csv'
test_dataset_dir = '/Users/ouyang/Documents/GitHub/ST7_FER_Projet/FER_Dataset/csv/test.csv'

print('Getting data. It may take a few minutes')
X_dev, X_res_dev, Y_dev   = get_data(dev_dataset_dir)
X_test, X_res_test, Y_test   = get_data(test_dataset_dir)
print('Import dataset successfully')

In [ ]:
BS = 128 # batch size,batch size = 128

from tensorflow.keras.preprocessing.image import ImageDataGenerator

def get_datagen(dataset, BS = 128, aug=False):
    if aug:
        datagen = ImageDataGenerator(
                            rescale=1./255,
                            featurewise_center=False,
                            featurewise_std_normalization=False,
                            rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.1,
                            horizontal_flip=True)
    else:
        datagen = ImageDataGenerator(rescale=1./255)

    return datagen.flow_from_directory(
            dataset,
            target_size=(48, 48),
            color_mode='grayscale',
            shuffle = True,
            class_mode='categorical',
            batch_size=BS)

dev_generator    = get_datagen("/Users/ouyang/Documents/GitHub/ST7_FER_Projet/FER_Dataset/test-private")
test_generator  = get_datagen("/Users/ouyang/Documents/GitHub/ST7_FER_Projet/FER_Dataset/test-public")

## Import models

In [ ]:
model1 = load_model('trained_models/Basic-EPOCHS_300-DROPOUT_0.3-test_acc_0.641.h5')
model2 = load_model('trained_models/Basic-EPOCHS_350-DROPOUT_0.3-test_acc_0.646.h5')

Resnet_model = load_model('trained_models/RESNET50-EPOCHS_100test_acc_0.716.h5')
Resnet_Aux_model_wcw = load_model("cs230-fer/RESNET50-WCW-AUX-BEST-72.4.h5")
Senet_Aux_model = load_model('cs230-fer/SENET50-AUX-BEST-72.5.h5')
Senet_Aux_model_wcw = load_model('cs230-fer/SENET50-WCW-AUX-BEST-71.6.h5')
VGG_Aux_model = load_model("trained_models/VGG16-EPOCHS_50test_acc_0.701.h5")

In [ ]:
print('\n', 'model 1', '\n','Evaluate on dev data')
results_dev = model1.evaluate(X_dev,Y_dev)
print('dev loss, dev acc:', results_dev)

print('\n# Evaluate on test data')
results_test = model1.evaluate(X_test,Y_test)
print('test loss, test acc:', results_test)

In [ ]:
print('\n', 'model 2', '\n','Evaluate on dev data')
results_dev = model2.evaluate(X_dev,Y_dev)
print('dev loss, dev acc:', results_dev)

print('\n# Evaluate on test data')
results_test = model2.evaluate(X_test,Y_test)
print('test loss, test acc:', results_test)

In [ ]:
print('\n','Resnet_model' , '\n', 'Evaluate on dev data')
results_dev = Resnet_model.evaluate(X_res_dev,Y_dev)
print('dev loss, dev acc:', results_dev)

print('\n Evaluate on test data')
results_test = Resnet_model.evaluate(X_res_test,Y_test)
print('test loss, test acc:', results_test)

In [ ]:
print('\n','Resnet_Aux_model_wcw' , '\n', 'Evaluate on dev data')
results_dev = Resnet_Aux_model_wcw.evaluate(X_res_dev,Y_dev)
print('dev loss, dev acc:', results_dev)

print('\n Evaluate on test data')
results_test = Resnet_Aux_model_wcw.evaluate(X_res_test,Y_test)
print('test loss, test acc:', results_test)

In [ ]:
print('\n','Senet_Aux_model' , '\n', 'Evaluate on dev data')
results_dev = Senet_Aux_model.evaluate(X_res_dev,Y_dev)
print('dev loss, dev acc:', results_dev)

print('\n Evaluate on test data')
results_test = Senet_Aux_model.evaluate(X_res_test,Y_test)
print('test loss, test acc:', results_test)

In [ ]:
print('\n','Senet_Aux_model_wcw' , '\n', 'Evaluate on dev data')
results_dev = Senet_Aux_model_wcw.evaluate(X_res_dev,Y_dev)
print('dev loss, dev acc:', results_dev)

print('\n Evaluate on test data')
results_test = Senet_Aux_model_wcw.evaluate(X_res_test,Y_test)
print('test loss, test acc:', results_test)

In [ ]:
print('\n','VGG_Aux_model' , '\n', 'Evaluate on dev data')
results_dev = VGG_Aux_model.evaluate(X_res_dev,Y_dev)
print('dev loss, dev acc:', results_dev)

print('\n Evaluate on test data')
results_test = VGG_Aux_model.evaluate(X_res_test,Y_test)
print('test loss, test acc:', results_test)

## Ensemble models

In [ ]:
models_SOA = [model2]
models_TL = [Resnet_model,Resnet_Aux_model_wcw, Senet_Aux_model, Senet_Aux_model_wcw, VGG_Aux_model] 

models = [models_SOA, models_TL]
#models_list = ['model1', 'model2', 'Resnet_model',' Resnet_Aux_model_wcw', 'Senet_Aux_model', 'Senet_Aux_model_wcw', 'VGG_Aux_model']

In soft voting, every individual classifier provides a probability value that a specific data point belongs to a particular target class. The predictions are weighted by the classifier's importance and summed up. Then the target label with the greatest sum of weighted probabilities wins the vote

[sklearn.metrics.accuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html)

In [ ]:
# make an ensemble prediction for multi-class classification
def ensemble_predictions(models_SOA, testX, models_TL, testresX):
  # make predictions
  yhats = np.zeros((len(models_SOA)+len(models_TL),testX.shape[0],7))

  for model_ind in range(len(models_SOA)):
    yhat = models_SOA[model_ind].predict(testX)
    yhats[model_ind,:,:] = yhat

  for model_ind in range(len(models_TL)):
    yhat = models_TL[model_ind].predict(testresX) # res = rescale
    yhats[len(models_SOA)+model_ind,:,:] = yhat

  summed = np.sum(yhats, axis=0)
  result = np.argmax(summed, axis=1)
  return result
 
# evaluate a specific number of members in an ensemble
def evaluate_n_members(models_SOA, testX, models_TL, testresX, testy):
    # select a subset of members
    #subset = members[:n_members]
    #print(len(subset))
    # make prediction
    yhat = ensemble_predictions(models_SOA, testX, models_TL, testresX)
    print(yhat)
    # calculate accuracy
    return accuracy_score(testy, yhat)

In [ ]:
ens_acc = evaluate_n_members(models_SOA, X_test, models_TL, X_res_test, np.argmax(Y_test, axis=1))
print(ens_acc)

## Demonstration

In [ ]:
import cv2

# Load an color image in grayscale
img = cv2.imread('/Users/ouyang/Documents/GitHub/ST7_FER_Projet/FER_Dataset/demo/0 anrgy/WechatIMG1773.jpeg', 0)
print(np.shape(img))

In [ ]:
def demo(demo_path, models_SOA, models_TL, Resize_pixelsize = 197):
    
    file_stream = file_io.FileIO(dataset_path, mode='r')
    data = pd.read_csv(file_stream)
    data['pixels'] = data['pixels'].apply(lambda x: [int(pixel) for pixel in x.split()])
    X, Y = data['pixels'].tolist(), data['emotion'].values
    X = np.array(X, dtype='float32').reshape(-1,48,48,1)
    X = X/255.0
   
    X_res = np.zeros((X.shape[0], Resize_pixelsize,Resize_pixelsize,3))
    for ind in range(X.shape[0]): 
        sample = X[ind]
        sample = sample.reshape(48, 48)
        image_resized = resize(sample, (Resize_pixelsize, Resize_pixelsize), anti_aliasing=True)
        X_res[ind,:,:,:] = image_resized.reshape(Resize_pixelsize,Resize_pixelsize,1)   
    
    return  ensemble_predictions(demo_path, models_SOA, X, models_TL, X_res)

In [ ]:
print(demo(demo_path, models_SOA, models_TL))

In [ ]:
emotions = {0:'Angry', 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise', 6:'Neutral'}

model = load_model(r'/Users/ouyang/Documents/GitHub/ST7_FER_Projet/ST7_FER_Github/ST7_models/cs230-fer/VGG16-AUX-BEST-70.2.h5') 
demo_generator  = get_datagen(r"/Users/ouyang/Documents/GitHub/ST7_FER_Projet/FER_Dataset/demo")

# y_true = demo_generator.classes
# print(y_true)
y_pred = model.predict(demo_generator)
